## 注意力分数

<img src = "photo/attention_score.png">

In [1]:
import torch
import math
from torch import nn
from d2l import torch as d2l

### 掩蔽SoftMax

softmax操作用于输出一个概率分布作为注意力权重。 在某些情况下，并非所有的值都应该被纳入到注意力汇聚中。 

例如，为了高效处理小批量数据集， 某些文本序列被填充了没有意义的特殊词元。 为了仅将有意义的词元作为值来获取注意力汇聚， 我们可以指定一个有效序列长度（即词元的个数）， 以便在计算softmax时过滤掉超出指定范围的位置。 

通过这种方式，我们可以在下面的masked_softmax函数中 实现这样的掩蔽softmax操作， 其中任何超出有效长度的位置都被掩蔽并置为0。

In [2]:
def masked_softmax(X,valid_lens):
    """
    

    Args:
        X (_type_): _description_
        valid_lens (_type_): _description_

    Returns:
        _type_: _description_
    """
    if valid_lens is None:
        return nn.functional.softmax(X, dim=-1)     # dim=-1相当于最后一个维度，相当于每一行进行操作：https://zhuanlan.zhihu.com/p/525276061
    else:
        shape = X.shape
        if valid_lens.dim() == 1:       # 如果维度是1
            valid_lens = torch.repeat_interleave(valid_lens,shape[1])
        else:
            valid_lens = valid_lens.reshape(-1)     # 相当于展平成一维
        
        # 最后一个轴上被隐蔽的元素使用一个非常大的负值进行替换，从而其softmax输出为0
        X = d2l.sequence_mask(X.reshape(-1, shape[-1]), valid_lens,
                              value=-1e6)
        return nn.functional.softmax(X.reshape(shape),dim=-1)

In [3]:
masked_softmax(torch.rand(2, 2, 4), torch.tensor([2, 3]))

tensor([[[0.5442, 0.4558, 0.0000, 0.0000],
         [0.5351, 0.4649, 0.0000, 0.0000]],

        [[0.3948, 0.2849, 0.3203, 0.0000],
         [0.2800, 0.4219, 0.2980, 0.0000]]])

第一个批量，前两列元素是有效的，后面的全是0

第二个批量，前散列元素是有效的，后面的全是0

In [4]:
masked_softmax(torch.rand(2, 2, 4), torch.tensor([[1, 3], [2, 4]]))

tensor([[[1.0000, 0.0000, 0.0000, 0.0000],
         [0.3294, 0.2930, 0.3776, 0.0000]],

        [[0.6012, 0.3988, 0.0000, 0.0000],
         [0.1290, 0.2965, 0.2559, 0.3185]]])

### 加性注意力

如上文图所示：加性注意力

$$a(\mathbf q, \mathbf k) = \mathbf w_v^\top \text{tanh}(\mathbf W_q\mathbf q + \mathbf W_k \mathbf k) \in \mathbb{R},$$

In [5]:
class AdditiveAttention(nn.Module):
    
    def __init__(self,key_size, query_size, num_hiddens, drop_out, **kwargs) -> None:
        super(AdditiveAttention).__init__()